# Import libraries and data

In [182]:
# import libraries 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

from Bio import SeqIO
from utils import protein_analysis

In [183]:
data_path = os.getcwd() + "/Datasets/"

## Curated CSF 1000+ proteins data set

In [184]:
csf = pd.read_csv(data_path + "CSF/all_csf_1000plus.csv")
csf

,Uniprot,#Peptides_Macron2018A,#Peptides_Macron2020,#Peptides_Zhang2015,#Peptides_Guldbrandsen2014,#Peptides_Macron2018B,#Peptides_Schutzer2010,#Peptides_Pan2007
0,Q6K0P9,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Q9GZZ8,1.0,3.0,NaN,NaN,NaN,NaN,NaN
2,P09529,3.0,3.0,4.0,4.0,1.0,NaN,NaN
3,P61019,2.0,3.0,NaN,NaN,2.0,NaN,NaN
4,Q9GZX9,4.0,4.0,3.0,3.0,4.0,5.0,1.0
...,...,...,...,...,...,...,...,...
5719,Q9ULJ1,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5720,Q9BTA9,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5721,Q86VF7,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5722,Q8NDV3,NaN,NaN,NaN,NaN,NaN,NaN,1.0


## Elevated brain proteome Uniprot sequences

In [185]:
#### TO DO ####
# create tsv instead of csv file

df = pd.read_csv(data_path + "Brain/Human_brain_elevated_Uniprot_seq.csv", sep=";", header=0, names=["Uniprot", "Sequence"]) 
# drop entries without sequence (obsolete)
df.dropna(subset=["Sequence"], inplace=True) # 3 entries dropped
# drop entries with non-standard amino acids
df = df[df["Sequence"].str.contains("B|U|X") == False] # 1 entry dropped
df

,Uniprot,Sequence
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...
...,...,...
2541,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...
2542,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...
2543,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...
2544,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...


# Feature generation

In [186]:
def get_uniprot(string):
    try:
        _, uniprot, _ = string.split("|")
    except:
        _, uniprot, _ = string.split("_", maxsplit=2)  
    return uniprot

def get_value(string):
    _, value = string.split("=")
    return float(value)

def read_uniprot_list(file):
    file = open(data_path + "Features/" + file, "r")
    lines = file.readlines()
    uniprots = []
    
    for line in lines:
        line_strip = line.strip()
        uniprots.append(line_strip)
        
    return uniprots  

## Sequence length

In [187]:
df["Length"] = df["Sequence"].apply(len)
df

,Uniprot,Sequence,Length
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422
...,...,...,...
2541,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212
2542,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420
2543,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548
2544,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406


## Amino acid composition & attributes

In [188]:
# df = df.apply(protein_analysis, seq_col="Sequence", axis=1)

In [189]:
# save or load dataframe
# df.to_csv(data_path + "Features/df_features_PA_brain.csv", index=False)
df = pd.read_csv(data_path + "Features/df_features_PA_brain.csv") 
df

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,Polarity_large,Polarizability_low,Polarizability_medium,Polarizability_large,Charge_positive,Charge_neutral,Charge_negative,Buried,Exposed,Intermediate
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,0.330334,0.332905,0.425450,0.241645,0.134961,0.755784,0.109254,0.376607,0.327763,0.326478
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,0.213355,0.281759,0.457655,0.260586,0.068404,0.856678,0.074919,0.545603,0.224756,0.250814
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,0.417785,0.315436,0.453020,0.231544,0.140940,0.713087,0.145973,0.354027,0.379195,0.255034
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0.179487,0.423077,0.307692,0.269231,0.115385,0.858974,0.025641,0.602564,0.089744,0.217949
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0.393365,0.277251,0.443128,0.279621,0.151659,0.703791,0.144550,0.412322,0.402844,0.215640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0.320132,0.290429,0.417492,0.292079,0.099835,0.806106,0.094059,0.439769,0.292904,0.272277
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0.373810,0.278571,0.452381,0.269048,0.116667,0.742857,0.140476,0.380952,0.342857,0.266667
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0.226277,0.328467,0.432482,0.239051,0.080292,0.826642,0.093066,0.549270,0.202555,0.235401
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0.399015,0.288177,0.423645,0.288177,0.133005,0.724138,0.142857,0.376847,0.366995,0.266010


## Structural features (NetSurfP-2.0)

In [190]:
nsp_features = pd.read_csv(data_path + "Features/features_human_proteome_no_filtering.csv")
nsp_features = nsp_features[["id", "disorder", "helix", "turn", "sheet"]]
nsp_features.columns = ["Uniprot", "Disorder_NSP", "Helix_NSP", "Turn_NSP", "Sheet_NSP"]

# nsp_features = nsp_features[["id", "disorder", "helix", "turn", "sheet", "Glycosylation_all"]]
# nsp_features.columns = ["Uniprot", "Disorder_NSP", "Helix_NSP", "Turn_NSP", "Sheet_NSP", "Glycosylation_all"]

In [191]:
# add structural features to feature dataframe
df = df.merge(nsp_features, on="Uniprot", how="inner")
df

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,Charge_positive,Charge_neutral,Charge_negative,Buried,Exposed,Intermediate,Disorder_NSP,Helix_NSP,Turn_NSP,Sheet_NSP
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,0.134961,0.755784,0.109254,0.376607,0.327763,0.326478,0.404884,0.241645,0.641388,0.116967
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,0.068404,0.856678,0.074919,0.545603,0.224756,0.250814,0.050489,0.700326,0.289902,0.009772
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,0.140940,0.713087,0.145973,0.354027,0.379195,0.255034,0.320470,0.498322,0.473154,0.028523
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0.115385,0.858974,0.025641,0.602564,0.089744,0.217949,0.038462,0.692308,0.307692,0.000000
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0.151659,0.703791,0.144550,0.412322,0.402844,0.215640,0.343602,0.080569,0.625592,0.293839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0.099835,0.806106,0.094059,0.439769,0.292904,0.272277,0.127888,0.338284,0.520627,0.141089
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0.116667,0.742857,0.140476,0.380952,0.342857,0.266667,0.261905,0.247619,0.588095,0.164286
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0.080292,0.826642,0.093066,0.549270,0.202555,0.235401,0.133212,0.682482,0.317518,0.000000
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0.133005,0.724138,0.142857,0.376847,0.366995,0.266010,0.206897,0.248768,0.571429,0.179803


## Solubility

In [192]:
weights = {'A': 0.8356471476582918,
           'C': 0.5208088354857734,
           'U': 0.5208088354857734, 
           'E': 0.9876987431418378,
           'D': 0.9079044671339564,
           'G': 0.7997168496420723,
           'F': 0.5849790194237692,
           'I': 0.6784124413866582,
           'H': 0.8947913996466419,
           'K': 0.9267104557513497,
           'M': 0.6296623675420369,
           'L': 0.6554221515081433,
           'N': 0.8597433107431216,
           'Q': 0.789434648348208,
           'P': 0.8235328714705341,
           'S': 0.7440908318492778,
           'R': 0.7712466317693457,
           'T': 0.8096922697856334,
           'W': 0.6374678690957594,
           'V': 0.7357837119163659,
           'Y': 0.6112801822947587}
A = 81.0581
B = -62.7775

def sol(seq):
    SWI = np.mean(([weights[i] for i in seq]))
    sol = 1/(1 + np.exp(-(81.0581*SWI + -62.7775)))
    return sol

In [193]:
df["Solubility"] = df["Sequence"].apply(sol)
df

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,Charge_neutral,Charge_negative,Buried,Exposed,Intermediate,Disorder_NSP,Helix_NSP,Turn_NSP,Sheet_NSP,Solubility
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,0.755784,0.109254,0.376607,0.327763,0.326478,0.404884,0.241645,0.641388,0.116967,0.698977
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,0.856678,0.074919,0.545603,0.224756,0.250814,0.050489,0.700326,0.289902,0.009772,0.062436
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,0.713087,0.145973,0.354027,0.379195,0.255034,0.320470,0.498322,0.473154,0.028523,0.827648
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0.858974,0.025641,0.602564,0.089744,0.217949,0.038462,0.692308,0.307692,0.000000,0.096285
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0.703791,0.144550,0.412322,0.402844,0.215640,0.343602,0.080569,0.625592,0.293839,0.849031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0.806106,0.094059,0.439769,0.292904,0.272277,0.127888,0.338284,0.520627,0.141089,0.351031
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0.742857,0.140476,0.380952,0.342857,0.266667,0.261905,0.247619,0.588095,0.164286,0.543458
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0.826642,0.093066,0.549270,0.202555,0.235401,0.133212,0.682482,0.317518,0.000000,0.156975
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0.724138,0.142857,0.376847,0.366995,0.266010,0.206897,0.248768,0.571429,0.179803,0.787281


## Transmembrane prediction

In [194]:
tmhmm = pd.read_csv(data_path + "Features/TMHMM_results_brain_proteome.txt", header=None, sep="\t", 
    names=["Uniprot", "Length", "ExpAA", "First60ExpAA", "PredHel", "Topology"])

In [195]:
# retrieve Uniprot ID
tmhmm["Uniprot"] = tmhmm["Uniprot"].apply(get_uniprot)
tmhmm["Length"] = tmhmm["Length"].apply(get_value)
tmhmm["ExpAA"] = tmhmm["ExpAA"].apply(get_value)
tmhmm["First60ExpAA"] = tmhmm["First60ExpAA"].apply(get_value)
tmhmm["PredHel"] = tmhmm["PredHel"].apply(get_value)
tmhmm

,Uniprot,Length,ExpAA,First60ExpAA,PredHel,Topology
0,Q9BZM6,244.0,18.75,1.12,1.0,Topology=o226-243i
1,P19086,355.0,0.13,0.00,0.0,Topology=o
2,Q9Y342,182.0,87.99,20.28,4.0,Topology=i37-56o66-88i100-122o142-164i
3,Q8TCC7,542.0,201.47,16.13,8.0,Topology=o15-37i127-144o177-199i211-233o243-26...
4,P58400,472.0,40.01,18.14,2.0,Topology=i31-53o396-418i
...,...,...,...,...,...,...
2499,Q6Q759,2223.0,0.02,0.00,0.0,Topology=o
2500,Q9NRZ5,378.0,76.73,24.79,3.0,Topology=o20-42i308-330o335-357i
2501,Q5JVS0,413.0,0.01,0.01,0.0,Topology=o
2502,Q3SXZ7,439.0,0.26,0.00,0.0,Topology=o


In [196]:
#### TO DO ####
# check why inner merge leads to missing entries
# why do we not have TMHMM predictions for all brain proteins?
# convert PredHel to binary value (more informative)

df = df.merge(tmhmm[["Uniprot", "ExpAA", "First60ExpAA", "PredHel"]], on="Uniprot", how="left")
df.fillna(0, inplace=True)
df["PredHel_binary"] = np.where(df["PredHel"] == 0, 0, 1)
df

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,Intermediate,Disorder_NSP,Helix_NSP,Turn_NSP,Sheet_NSP,Solubility,ExpAA,First60ExpAA,PredHel,PredHel_binary
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,0.326478,0.404884,0.241645,0.641388,0.116967,0.698977,0.86,0.00,0.0,0
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,0.250814,0.050489,0.700326,0.289902,0.009772,0.062436,0.00,0.00,0.0,0
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,0.255034,0.320470,0.498322,0.473154,0.028523,0.827648,0.00,0.00,0.0,0
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0.217949,0.038462,0.692308,0.307692,0.000000,0.096285,29.94,20.12,0.0,0
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0.215640,0.343602,0.080569,0.625592,0.293839,0.849031,22.93,3.09,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0.272277,0.127888,0.338284,0.520627,0.141089,0.351031,0.00,0.00,0.0,0
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0.266667,0.261905,0.247619,0.588095,0.164286,0.543458,0.00,0.00,0.0,0
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0.235401,0.133212,0.682482,0.317518,0.000000,0.156975,245.12,0.00,12.0,1
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0.266010,0.206897,0.248768,0.571429,0.179803,0.787281,0.02,0.01,0.0,0


## Subcellular location prediction

In [197]:
deeploc = pd.read_csv(data_path + "Features/DeepLoc_results_human_proteome.txt", sep="\t")
deeploc.rename(columns={"ID":"Uniprot"}, inplace=True)

# retrieve Uniprot ID
deeploc["Uniprot"] = deeploc["Uniprot"].apply(get_uniprot)
deeploc

,Uniprot,Location,Membrane,Nucleus,Cytoplasm,Extracellular,Mitochondrion,Cell_membrane,Endoplasmic_reticulum,Plastid,Golgi_apparatus,Lysosome/Vacuole,Peroxisome
0,Q8WZ42,Cytoplasm,0.0054,0.0528,0.9364,0.0001,0.0000,0.0090,0.0001,0.0001,0.0002,0.0008,0.0006
1,Q8WXI7,Cytoplasm,0.2836,0.1055,0.6656,0.0005,0.0002,0.2102,0.0101,0.0001,0.0041,0.0026,0.0012
2,Q8NF91,Cytoplasm,0.1989,0.0707,0.5495,0.0002,0.0008,0.0319,0.0229,0.0001,0.2861,0.0377,0.0001
3,Q7Z5P9,Extracellular,0.0550,0.0407,0.1452,0.7810,0.0033,0.0277,0.0007,0.0000,0.0002,0.0011,0.0000
4,Q5VST9,Cytoplasm,0.0108,0.0871,0.8976,0.0002,0.0000,0.0140,0.0001,0.0000,0.0002,0.0003,0.0005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20371,P02729,Extracellular,0.0006,0.0031,0.0268,0.9364,0.0315,0.0000,0.0000,0.0022,0.0000,0.0001,0.0000
20372,P0DOY5,Extracellular,0.1216,0.0002,0.0047,0.7318,0.1119,0.0346,0.0005,0.0144,0.0011,0.1006,0.0002
20373,P01858,Extracellular,0.0399,0.2627,0.0592,0.6342,0.0264,0.0119,0.0008,0.0000,0.0047,0.0000,0.0000
20374,P0DPI4,Extracellular,0.3387,0.0396,0.0151,0.4207,0.1285,0.2067,0.0033,0.0027,0.0677,0.1158,0.0000


In [198]:
deeploc["Location"].value_counts(dropna=False)

Cytoplasm                5602
Nucleus                  5430
Cell_membrane            3538
Extracellular            2034
Mitochondrion            1563
Endoplasmic_reticulum    1342
Golgi_apparatus           454
Peroxisome                160
Lysosome/Vacuole          136
Plastid                   117
Name: Location, dtype: int64

In [199]:
# add subcellular locations as binary features
for i in deeploc["Location"].unique():
    deeploc_subset = deeploc[deeploc["Location"] == i]
    df[i] = np.where(df["Uniprot"].isin(deeploc_subset["Uniprot"]), 1, 0)

In [200]:
for i in deeploc["Location"].unique():
    print(df[i].value_counts())

0    1898
1     644
Name: Cytoplasm, dtype: int64
0    2332
1     210
Name: Extracellular, dtype: int64
0    2521
1      21
Name: Lysosome/Vacuole, dtype: int64
0    2000
1     542
Name: Nucleus, dtype: int64
0    1697
1     845
Name: Cell_membrane, dtype: int64
0    2428
1     114
Name: Endoplasmic_reticulum, dtype: int64
0    2466
1      76
Name: Mitochondrion, dtype: int64
0    2471
1      71
Name: Golgi_apparatus, dtype: int64
0    2531
1      11
Name: Peroxisome, dtype: int64
0    2534
1       8
Name: Plastid, dtype: int64


##  Domains

### Cadherin-1 (PS00232)

In [201]:
PS00232 = read_uniprot_list("PS00232.txt")
df["PS00232"] = np.where(df["Uniprot"].isin(PS00232), 1, 0)
df["PS00232"].value_counts()

0    2496
1      46
Name: PS00232, dtype: int64

### G-protein receptor F1 (PS00237)

In [202]:
PS00237 = read_uniprot_list("PS00237.txt")
df["PS00237"] = np.where(df["Uniprot"].isin(PS00237), 1, 0)
df["PS00237"].value_counts()

0    2471
1      71
Name: PS00237, dtype: int64

### Homeobox (PS00027)

In [203]:
PS00027 = read_uniprot_list("PS00027.txt")
df["PS00027"] = np.where(df["Uniprot"].isin(PS00027), 1, 0)
df["PS00027"].value_counts()

0    2494
1      48
Name: PS00027, dtype: int64

### Zinc Finger C2H2 (PS00028)

In [204]:
PS00028 = read_uniprot_list("PS00028.txt")
df["PS00028"] = np.where(df["Uniprot"].isin(PS00028), 1, 0)
df["PS00028"].value_counts()

0    2501
1      41
Name: PS00028, dtype: int64

### EGF1 (PS00022)

In [205]:
PS00022 = read_uniprot_list("PS00022.txt")
df["PS00022"] = np.where(df["Uniprot"].isin(PS00022), 1, 0)
df["PS00022"].value_counts()

0    2501
1      41
Name: PS00022, dtype: int64

### EGF2 (PS01186)

In [206]:
PS01186 = read_uniprot_list("PS01186.txt")
df["PS01186"] = np.where(df["Uniprot"].isin(PS01186), 1, 0)
df["PS01186"].value_counts()

0    2506
1      36
Name: PS01186, dtype: int64

## Glycosylation prediction

### NetOglyc

In [207]:
# oglyc = pd.read_csv(data_path + "Features/prediction_results.txt")

### NetNglyc

In [208]:
def netNglyc_filter(file, file_name):
    """
    """
    # open results file of netNglyc predictions
    results = open(file, "r")
    lines = results.readlines()
    
    # open new file to save filtered lines to
    filtered_results = open(data_path + "Features/" + file_name + ".txt", "w+")
    
    for line in lines:
        # save relevant lines to new file
        if line[:3] == "sp|":
            filtered_results.writelines(line)
    
    # close file
    filtered_results.close()
    
    return None
    
def split_netNglyc(df):
    """
    """
    string = df[0]
    
    # retrieve information from first column
    name, pos, seq = string.split()

    # retrieve Uniprot ID from description
    uniprot = get_uniprot(name)
    
    df["Uniprot"] = uniprot
    df["Position"] = pos
    df["Sequence"] = seq
    
    # drop old column
    df.drop(columns=[0], axis=1, inplace=True)
    
    # reorder columns
    df = df[["Uniprot", "Position", "Sequence", "Potential", "Jury agreement", "Result"]]
    
    return df

In [209]:
# # filter netNglyc results file
# netNglyc_filter(data_path + "Features/NetNglyc_results_human_proteome.out", "NetNglyc_results_human_proteome_filtered")

In [210]:
# # create clean dataframe of glycosylation prediction results
# netnglyc = pd.read_csv(data_path + "Features/NetNglyc_results_human_proteome_filtered.txt", sep="\t", header=None) 
# netnglyc.dropna(axis=1, how="all", inplace=True)
# netnglyc.columns = [0, "Potential", "Jury agreement", "Result"]
# netnglyc = netnglyc.apply(split_netNglyc, axis=1)
# netnglyc

In [211]:
# save or load dataframe
# netnglyc.to_csv(data_path + "Features/NetNglyc.csv", index=False)
netnglyc = pd.read_csv(data_path + "Features/NetNglyc.csv") 

In [212]:
# filter for predicted glycosylation sites, "-" means predicted negative site
netnglyc_pos_1 = netnglyc[netnglyc["Result"].str.contains("+", regex=False)]
netnglyc_pos_2 = netnglyc[netnglyc["Result"].str.contains("++", regex=False)]
netnglyc_pos_3 = netnglyc[netnglyc["Result"].str.contains("+++", regex=False)]
print(len(netnglyc_pos_1))
print(len(netnglyc_pos_2))
print(len(netnglyc_pos_3))

16983
7168
1321


In [213]:
glyc_sites_1 = pd.DataFrame(netnglyc_pos_1["Uniprot"].value_counts(), index=None).reset_index()
glyc_sites_1.columns = ["Uniprot", "Glycosylation_*"]
glyc_sites_2 = pd.DataFrame(netnglyc_pos_2["Uniprot"].value_counts(), index=None).reset_index()
glyc_sites_2.columns = ["Uniprot", "Glycosylation_**"]
glyc_sites_3 = pd.DataFrame(netnglyc_pos_3["Uniprot"].value_counts(), index=None).reset_index()
glyc_sites_3.columns = ["Uniprot", "Glycosylation_***"]

In [214]:
# # add glycosylation as a binary feature
# df["Glycosylation"] = np.where(df["Uniprot"].isin(glyc_sites["Uniprot"]), 1, 0)

# add number of glycosylation sites as a feature
df = df.merge(glyc_sites_1, on="Uniprot", how="left")
df = df.merge(glyc_sites_2, on="Uniprot", how="left")
df = df.merge(glyc_sites_3, on="Uniprot", how="left")
df.fillna(0, inplace=True)
df

#### TO DO ####
# add glycosylation sites at different confidence cut-off (+++, ++, +) as features

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,Plastid,PS00232,PS00237,PS00027,PS00028,PS00022,PS01186,Glycosylation_*,Glycosylation_**,Glycosylation_***
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,0,0,0,0,0,0,0,0.0,0.0,0.0
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,0,0,0,0,0,0,0,2.0,1.0,0.0
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,0,0,0,0,0,0,0,1.0,1.0,1.0
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0,0,0,0,0,0,0,2.0,2.0,1.0
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [215]:
df["Glycosylation_***"].value_counts()

0.0    2376
1.0     139
2.0      22
3.0       5
Name: Glycosylation_***, dtype: int64

In [216]:
df["Glycosylation_binary_*"] = np.where(df["Glycosylation_*"] == 0, 0, 1)
df["Glycosylation_binary_**"] = np.where(df["Glycosylation_**"] == 0, 0, 1)
df["Glycosylation_binary_***"] = np.where(df["Glycosylation_***"] == 0, 0, 1)

### GlycoMine

In [217]:
# datasets and thresholds taken from https://glycomine.erc.monash.edu/Lab/GlycoMine/
glycomine_n = pd.read_csv(data_path + "Features/GlycoMine_N_results.txt", sep=" ") 
glycomine_n_pos = glycomine_n[glycomine_n["Value"] > 0.5]

glycomine_o = pd.read_csv(data_path + "Features/GlycoMine_O_results.txt", sep=" ") 
glycomine_o_pos = glycomine_o[glycomine_o["Value"] > 0.502]

glycomine_c = pd.read_csv(data_path + "Features/GlycoMine_C_results.txt", sep=" ") 
glycomine_c_pos = glycomine_c[glycomine_c["Value"] > 0.555]

In [218]:
df["GlycoMine_N"] = np.where(df["Uniprot"].isin(set(glycomine_n_pos["UniProtID"])), 1, 0)
df["GlycoMine_O"] = np.where(df["Uniprot"].isin(set(glycomine_o_pos["UniProtID"])), 1, 0)
df["GlycoMine_C"] = np.where(df["Uniprot"].isin(set(glycomine_c_pos["UniProtID"])), 1, 0)

## GPI-Anchor prediction (NetGPI)

In [219]:
netgpi = pd.read_csv(data_path + "Features/NetGPI_results_brain_proteome.txt", sep="\t", header=1, 
    names=["Uniprot", "Length", "Result", "Omega-site", "Likelihood", "Amino acid"]) 

In [220]:
netgpi["Uniprot"] = netgpi["Uniprot"].apply(get_uniprot)

In [221]:
netgpi_pos = netgpi[netgpi["Result"] == "GPI-Anchored"]
netgpi_pos

,Uniprot,Length,Result,Omega-site,Likelihood,Amino acid
0,Q9BZM6,244,GPI-Anchored,222,0.324,A
59,Q96JJ6,628,GPI-Anchored,602,0.755,G
79,Q96CW9,530,GPI-Anchored,507,0.580,G
100,P23515,440,GPI-Anchored,417,0.343,S
116,Q12860,1018,GPI-Anchored,993,0.765,S
...,...,...,...,...,...,...
2230,Q9H4Q4,367,GPI-Anchored,334,0.366,S
2253,Q86UN2,441,GPI-Anchored,420,0.488,S
2335,O94779,1100,GPI-Anchored,1072,0.626,S
2344,O95971,181,GPI-Anchored,159,0.755,S


In [222]:
df["GPI-anchor"] = np.where(df["Uniprot"].isin(netgpi_pos["Uniprot"]), 1, 0)

## Signal peptide

In [223]:
#### TO DO ####
# include signal peptide probability as a continuous variable instead of binary feature

signalp = pd.read_csv(data_path + "Features/SignalP_results_human_proteome.txt", sep="\t", index_col=False, header=None, 
    skiprows=2, names=["Uniprot", "Prediction", "Likelihood-Other", "Likelihood-SP", "CS Position"])

# retrieve Uniprot ID
signalp["Uniprot"] = signalp["Uniprot"].apply(get_uniprot)
signalp_pos = signalp[signalp["Prediction"] == "SP"]
signalp_pos

,Uniprot,Prediction,Likelihood-Other,Likelihood-SP,CS Position
4,P22223,SP,0.000224,0.999762,CS pos: 24-25. Pr: 0.7575
5,Q9BXJ4,SP,0.000201,0.999764,CS pos: 22-23. Pr: 0.9800
6,P09871,SP,0.000226,0.999694,CS pos: 15-16. Pr: 0.9805
7,Q9ULX7,SP,0.000220,0.999737,CS pos: 18-19. Pr: 0.9241
26,Q16787,SP,0.002463,0.997510,CS pos: 36-37. Pr: 0.7144
...,...,...,...,...,...
20348,Q9HBH1,SP,0.190636,0.809347,CS pos: 23-24. Pr: 0.5940
20358,P98172,SP,0.000280,0.999700,CS pos: 27-28. Pr: 0.9724
20363,P34910,SP,0.260518,0.739443,CS pos: 21-22. Pr: 0.4787
20365,Q96PL5,SP,0.048187,0.951777,CS pos: 29-30. Pr: 0.8838


In [224]:
df["Signal peptide"] = np.where(df["Uniprot"].isin(signalp_pos["Uniprot"]), 1, 0)
df["Signal peptide"].value_counts()

0    2014
1     528
Name: Signal peptide, dtype: int64

## Nucleotide-binding proteins

In [225]:
# downloaded from http://biomine.cs.vcu.edu/servers/DRNApred/
with open(data_path + "Features/DRNApred_RNA_Uniprot_human_proteome.txt") as f:  
    RNA_bind = f.read().splitlines() 
with open(data_path + "Features/DRNApred_DNA_Uniprot_human_proteome.txt") as f:  
    DNA_bind = f.read().splitlines() 

In [226]:
df["RNA_binding"] = np.where(df["Uniprot"].isin(RNA_bind), 1, 0)
df["DNA_binding"] = np.where(df["Uniprot"].isin(DNA_bind), 1, 0)

## Protein-binding prediction

In [227]:
def parse_SCRIBER_results(lines):
    
    uniprots = []
    preds = []
    
    for i in range(0, len(lines), 4):    
        
        # check if lines contains Uniprot header
        assert lines[i][0] == ">", lines[i]
        # add uniprot to list
        uniprot = lines[i][1:].strip()
        uniprots.append(uniprot)
        
        # get protein binding predictions
        prot_bind = lines[i+2].strip()
        # check if lines contains Uniprot header
        assert all(c in "01" for c in prot_bind), i
        # calculate fraction of protein-binding to list
        prot_bind_count = prot_bind.count("1")
        prot_bind_fraction = prot_bind_count / len(prot_bind)
        preds.append(prot_bind_fraction)
        
    return uniprots, preds

In [228]:
# downloaded from http://biomine.cs.vcu.edu/servers/SCRIBER/
with open(data_path + "Features/SCRIBER_results_human_proteome.txt") as f:  
    lines = f.readlines()

uniprots, preds = parse_SCRIBER_results(lines)

In [229]:
prot_bind_df = pd.DataFrame({"Uniprot":uniprots, "Prot_bind":preds})

In [230]:
df = df.merge(prot_bind_df, on="Uniprot", how="left")
df.fillna(0, inplace=True)
df["Prot_bind_binary"] = np.where(df["Prot_bind"] == 0, 0, 1)

## CSF presence (Label)

In [231]:
df["CSF"] = np.where(df["Uniprot"].isin(csf["Uniprot"]), 1, -1)
df

,Uniprot,Sequence,Length,Molecular weight,A,C,D,E,F,G,...,GlycoMine_N,GlycoMine_O,GlycoMine_C,GPI-anchor,Signal peptide,RNA_binding,DNA_binding,Prot_bind,Prot_bind_binary,CSF
0,Q8TDC3,MSSGAKEGGGGSPAYHLPHPHPHPPQHAQYVGPYRLEKTLGKGQTG...,778,85085.7493,0.051414,0.012853,0.043702,0.065553,0.029563,0.092545,...,1,1,0,0,0,0,0,0.029563,1,1
1,P48065,MDGKVAVQECGPPAVSWVPEEGEKLDQEDEDQVKDRGQWTNKMEFV...,614,69367.5176,0.052117,0.037459,0.030945,0.043974,0.083062,0.081433,...,1,0,0,0,0,0,0,0.340391,1,-1
2,Q9Y250,MGSVSSLISGHSFHSKHCRASQYKLRKSSHLKKLNRYSDGLLRFGF...,596,66612.1565,0.060403,0.011745,0.041946,0.104027,0.023490,0.072148,...,1,1,0,0,0,0,1,0.145973,1,-1
3,P0DMW5,MAASAALSAAAAAAALSGLAVRLSRSAAARGSYGAFCKGLTRTLLT...,78,8388.8176,0.230769,0.012821,0.012821,0.012821,0.064103,0.051282,...,0,0,0,0,0,0,0,1.000000,1,-1
4,P21579,MVSESHHEALAAPPVTTVATVLPSNATEPASPGEGKEDAFSKLKEK...,422,47572.5391,0.063981,0.014218,0.061611,0.082938,0.045024,0.059242,...,0,1,0,0,0,0,0,0.021327,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Q8IZU8,MALMFTGHLLFLALLMFAFSTFEESVSNYSEWAVFTDDIDQFKTQK...,1212,139235.8716,0.067657,0.009076,0.045380,0.048680,0.063531,0.055281,...,0,0,0,0,1,0,0,0.000000,0,-1
2538,Q4JDL3,MSSPRDFRAEPVNDYEGNDSEAEDLNFRETLPSSSQENTPRSKVFE...,420,48422.3273,0.047619,0.019048,0.057143,0.083333,0.052381,0.045238,...,0,0,0,0,0,0,0,0.321429,1,-1
2539,Q8N4V2,MEEDLFQLRQLPVVKFRRTGESARSEDDTASGEHEVQIEGVHVGLE...,548,60768.4670,0.085766,0.023723,0.031022,0.062044,0.056569,0.082117,...,0,0,0,0,0,0,0,0.472628,1,-1
2540,P48426,MATPGNLGSSVLASKTKTKKKHFVAQKVKLFRASDPLLSVLMWGVN...,406,46224.0440,0.056650,0.009852,0.068966,0.073892,0.046798,0.051724,...,0,1,0,0,0,0,0,0.009852,1,-1


In [232]:
df["CSF"].value_counts()

-1    1547
 1     995
Name: CSF, dtype: int64

In [233]:
df_pos = df[df["CSF"] == 1]
df_neg = df[df["CSF"] == -1]

In [234]:
df.corr()["CSF"].sort_values()[:20]

Nucleus                -0.246287
Helix_NSP              -0.226406
Isoelectric point      -0.175326
DNA_binding            -0.159341
L                      -0.152036
Prot_bind              -0.150457
Polarizability_large   -0.134748
Volume_large           -0.134748
Instability index      -0.131959
H                      -0.130285
F                      -0.114687
Hydrophobic            -0.112509
PS00237                -0.111486
M                      -0.106060
Polarity_low           -0.104891
Charge_neutral         -0.101597
PS00027                -0.099416
PS00028                -0.096285
Intermediate           -0.090137
W                      -0.090026
Name: CSF, dtype: float64

In [235]:
df.corr()["CSF"].sort_values()[-30:]

K                     0.070650
Turn_NSP              0.073415
Golgi_apparatus       0.079288
E                     0.089734
V                     0.090442
GPI-anchor            0.090854
Polarizability_low    0.095123
Polarity_large        0.098591
T                     0.099584
G                     0.102976
Glycosylation_*       0.118783
Glycosylation_**      0.120987
Polar                 0.124011
Cell_membrane         0.127036
N                     0.134897
PS01186               0.135807
Solubility            0.135965
PS00022               0.140455
PredHel_binary        0.145714
Exposed               0.146245
Charge_negative       0.148227
Extracellular         0.151664
GlycoMine_N           0.154729
PS00232               0.163227
D                     0.180639
Molecular weight      0.198641
Length                0.202620
Sheet_NSP             0.283703
Signal peptide        0.356304
CSF                   1.000000
Name: CSF, dtype: float64

In [236]:
df.columns

Index(['Uniprot', 'Sequence', 'Length', 'Molecular weight', 'A', 'C', 'D', 'E',
       'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V',
       'W', 'Y', 'Isoelectric point', 'Instability index', 'Polar', 'Neutral',
       'Hydrophobic', 'Volume_small', 'Volume_medium', 'Volume_large',
       'Polarity_low', 'Polarity_medium', 'Polarity_large',
       'Polarizability_low', 'Polarizability_medium', 'Polarizability_large',
       'Charge_positive', 'Charge_neutral', 'Charge_negative', 'Buried',
       'Exposed', 'Intermediate', 'Disorder_NSP', 'Helix_NSP', 'Turn_NSP',
       'Sheet_NSP', 'Solubility', 'ExpAA', 'First60ExpAA', 'PredHel',
       'PredHel_binary', 'Cytoplasm', 'Extracellular', 'Lysosome/Vacuole',
       'Nucleus', 'Cell_membrane', 'Endoplasmic_reticulum', 'Mitochondrion',
       'Golgi_apparatus', 'Peroxisome', 'Plastid', 'PS00232', 'PS00237',
       'PS00027', 'PS00028', 'PS00022', 'PS01186', 'Glycosylation_*',
       'Glycosylation_**', 'Glycosylation_

# Save feature dataframe

In [237]:
df.to_csv(data_path + "Features/df_features_brain.csv", index=False)